In [ ]:
import os

# Environment variables for Trino connection (provided by the platform)
TRINO_HOST = os.environ.get("TRINO_HOST")
TRINO_USER = os.environ.get("TRINO_USER")
TRINO_CATALOG = os.environ.get("TRINO_CATALOG")

# Install required libraries
%pip install -q trino==0.332.0 trino[sqlalchemy]
%pip install -q datasketch

# Connect to the Trino query engine using DBAPI
from trino.dbapi import connect

conn = connect(
    host=TRINO_HOST,
    user=TRINO_USER,
    catalog=TRINO_CATALOG,
)
cur = conn.cursor()


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:

from datetime import date

# 🚦 Set these to control the time window (strings "YYYY-MM-DD") or None for open-ended.
# Examples:
# start_date = "2025-09-01"; end_date = None         -> from 2025-09-01 to end
# start_date = "2025-09-01"; end_date = "2025-09-15" -> within inclusive window
# start_date = None; end_date = None                 -> all data
start_date = "2025-09-01"
end_date   = "2025-09-07"

# 🚦 New variable to control duplicate URL removal (default is to remove duplicates)
remove_duplicate_urls = True

def _normalize_date_str(d):
    """
    Accepts a value that should be either None or a 'YYYY-MM-DD' string.
    - Returns None if input is None or empty after stripping.
    - Returns ISO 'YYYY-MM-DD' string if valid.
    - Raises ValueError if invalid format.
    """
    if d is None:
        return None
    s = str(d).strip().strip('"').strip("'")
    if not s:
        return None
    try:
        # Validate format and return normalized ISO string
        return date.fromisoformat(s).isoformat()
    except Exception:
        raise ValueError(f"Invalid date: {d!r}. Expected format 'YYYY-MM-DD' (e.g., '2025-09-01').")

START_DATE = _normalize_date_str(start_date)
END_DATE   = _normalize_date_str(end_date)
print("Using date filter:",
      f"start_date={START_DATE}" if START_DATE else "start_date=None",
      f"end_date={END_DATE}" if END_DATE else "end_date=None", sep=" | ")
print("Remove duplicate URLs:", remove_duplicate_urls)


Using date filter: | start_date=2025-09-01 | end_date=2025-09-07
Remove duplicate URLs: True


In [ ]:


# Define the schema and table to fetch data from
schema_name = "news"
table_name  = "collected_news"

# Build SQL query safely based on optional START_DATE / END_DATE.
# created_at is ISO8601, e.g., "2025-01-21T02:26:32Z".
# We filter by the DATE portion via substr(created_at, 1, 10) = 'YYYY-MM-DD'.
if START_DATE and END_DATE:
    query = (
        f"SELECT url, text FROM {schema_name}.{table_name} "
        f"WHERE substr(created_at, 1, 10) >= '{START_DATE}' "
        f"  AND substr(created_at, 1, 10) <= '{END_DATE}'"
    )
elif START_DATE:
    query = (
        f"SELECT url, text FROM {schema_name}.{table_name} "
        f"WHERE substr(created_at, 1, 10) >= '{START_DATE}'"
    )
elif END_DATE:
    query = (
        f"SELECT url, text FROM {schema_name}.{table_name} "
        f"WHERE substr(created_at, 1, 10) <= '{END_DATE}'"
    )
else:
    query = f"SELECT url, text FROM {schema_name}.{table_name}"

# Execute the query and fetch results
cur.execute(query)
rows = cur.fetchall()

print(f"Retrieved {len(rows)} rows from {schema_name}.{table_name}.")
if rows:
    print("Sample row:", rows[0])


Retrieved 16019 rows from news.collected_news.
Sample row: ['https://www.express.co.uk/life-style/garden/2103924/families-urged-take-immediate-action-blackbirds-visit-gardens', 'Families are being asked to take immediate action in their gardens as the deadly mosquito-borne Usutu virus threatens to wipe out blackbird populations across the country. Ecologists are pleading with families to make small changes that could significantly help protect one of our most beloved garden birds.\n\nFigures first revealed in June showed a 40% decline in blackbirds in Greater London since the virus was first detected in 2020 - and with climate change helping mosquitoes to expand their territory, experts warn that the problem is only set to worsen. The Usutu virus, which is spread by the Culex pipiens mosquito - also known as the house mosquito - does not harm humans in the UK, but often proves lethal to birds - especially blackbirds, who are particularly prone to infection.\n\nWith longer summers, risi

In [ ]:

import pandas as pd

# Optionally remove duplicate URLs before analysis
if remove_duplicate_urls:
    df = pd.DataFrame(rows, columns=["url", "text"])
    before_count = len(df)
    df = df.drop_duplicates(subset="url", keep="first")
    after_count = len(df)
    removed = before_count - after_count
    print(f"Removed {removed} duplicate URLs (from {before_count} records).")
    # Update rows to the deduplicated list of (url, text) pairs
    rows = df[["url", "text"]].values.tolist()

# Arrange data in a dictionary: {id: text}, where id is the URL
articles_dict = {url: text for url, text in rows}
print(f"Number of articles loaded into dictionary: {len(articles_dict)}")

# Prepare a list of (id, text) tuples for analysis
articles_list = list(articles_dict.items())


Removed 5184 duplicate URLs (from 16019 records).
Number of articles loaded into dictionary: 10835


In [ ]:

from datasketch import MinHash, MinHashLSH

class NearDuplicateDetector:
    """
    Language-independent near-duplicate detection using character shingles + MinHash + LSH.
    """
    def __init__(self, threshold: float = 0.8, num_perm: int = 128, shingle_size: int = 5):
        self.threshold = float(threshold)
        self.num_perm = int(num_perm)
        self.shingle_size = int(shingle_size)
        self._lsh = MinHashLSH(threshold=self.threshold, num_perm=self.num_perm)
        self._minhashes = {}  # url -> MinHash signature

    def _preprocess(self, text: str) -> str:
        # Normalize whitespace and lowercase for robustness
        cleaned = ' '.join((text or "").split())
        return cleaned.lower()

    def _get_shingles(self, text: str) -> set:
        # Character shingles of length shingle_size
        cleaned_text = self._preprocess(text)
        n = self.shingle_size
        if len(cleaned_text) < n:
            return {cleaned_text}
        return {cleaned_text[i:i+n] for i in range(len(cleaned_text) - n + 1)}

    def _minhash_signature(self, shingles: set) -> MinHash:
        m = MinHash(num_perm=self.num_perm)
        for sh in shingles:
            m.update(sh.encode("utf-8"))
        return m

    def add_article(self, url: str, text: str) -> list:
        """
        Adds one article and returns list of prior URLs similar to it (>= threshold).
        """
        shingles = self._get_shingles(text)
        m = self._minhash_signature(shingles)
        similar_urls = []
        if self._minhashes:
            candidates = self._lsh.query(m)  # approximate neighborhood
            for cand_url in candidates:
                # Compute exact Jaccard similarity for candidates
                sim = m.jaccard(self._minhashes[cand_url])
                if sim >= self.threshold:
                    similar_urls.append(cand_url)
        # Insert after querying to avoid self-matching
        self._lsh.insert(url, m)
        self._minhashes[url] = m
        return similar_urls

    def add_articles(self, articles: list) -> dict:
        """
        Batch add [(url, text), ...]. Returns {url: [similar_urls]} for each URL that had matches.
        """
        duplicates_found = {}
        for url, text in articles:
            similar = self.add_article(url, text)
            if similar:
                duplicates_found[url] = similar
        return duplicates_found


In [ ]:

# Configure similarity threshold & shingling
THRESHOLD = 0.9   # adjust as needed (e.g., 0.85 or 0.9 for stricter matching)
NUM_PERM  = 128
SHINGLE_N = 5

detector = NearDuplicateDetector(threshold=THRESHOLD, num_perm=NUM_PERM, shingle_size=SHINGLE_N)
print(f"Detector initialized: threshold={THRESHOLD}, num_perm={NUM_PERM}, shingle_size={SHINGLE_N}")

# Batch process articles to find near-duplicates
duplicates_found = detector.add_articles(articles_list)

print(f"Articles with at least one near-duplicate: {len(duplicates_found)}")
# Optional peek at one example
for art_id, dup_ids in list(duplicates_found.items())[:5]:
    print(f"- {art_id} ~ {dup_ids}")
    break


Detector initialized: threshold=0.9, num_perm=128, shingle_size=5
Articles with at least one near-duplicate: 3903
- https://www.bournemouthecho.co.uk/news/national/25446659.farage-ends-conference-unity-plea-rowing-back-small-boats-pledge/ ~ ['https://www.edp24.co.uk/news/national/25446659.farage-ends-conference-unity-plea-rowing-back-small-boats-pledge/']


In [ ]:
import pandas as pd  # (Pandas already imported above, but included here for clarity)

# Build list of undirected unique pairs and compute similarities
pairs = []
pairs_with_sim = []
for new_id, similar_ids in duplicates_found.items():
    for existing_id in similar_ids:
        if new_id == existing_id:
            # Skip self-duplicates (should not happen if duplicates were removed)
            continue
        # Sort the pair for uniqueness (undirected pair)
        pair_sorted = tuple(sorted((existing_id, new_id)))
        pairs.append(pair_sorted)
        sim = detector._minhashes[new_id].jaccard(detector._minhashes[existing_id])
        pairs_with_sim.append((pair_sorted[0], pair_sorted[1], sim))

# Deduplicate pairs (ensure each pair is only listed once)
pairs = sorted(set(pairs))

# Build DataFrame of duplicate pairs with similarity
duplicates_df = pd.DataFrame(pairs_with_sim, columns=["id_1", "id_2", "similarity"])
# If any duplicate rows exist (unlikely after set), drop them and keep the highest similarity
duplicates_df = (duplicates_df
                 .sort_values(["id_1", "id_2", "similarity"], ascending=[True, True, False])
                 .drop_duplicates(subset=["id_1", "id_2"])
                 .reset_index(drop=True)
                )

print(f"Total unique near-duplicate pairs: {len(duplicates_df)}")
display(duplicates_df.head(10))



Total unique near-duplicate pairs: 64017


,id_1,id_2,similarity
0,http://maremmanews.it/post/allarme-oms-un-adol...,https://www.imgpress.it/culture/un-adolescente...,0.984375
1,http://stranieriinitalia.it/attualita/il-papa-...,https://stranieriinitalia.it/attualita/il-papa...,1.000000
2,http://stranieriinitalia.it/attualita/nave-ong...,https://stranieriinitalia.it/attualita/nave-on...,1.000000
3,http://stranieriinitalia.it/attualita/prato-co...,https://stranieriinitalia.it/attualita/prato-c...,1.000000
4,http://www.az-online.de/boulevard/der-skandals...,http://www.hna.de/leute/der-skandalstar-ist-la...,0.992188
5,http://www.az-online.de/boulevard/der-skandals...,http://www.merkur.de/boulevard/der-skandalstar...,0.992188
6,http://www.az-online.de/boulevard/der-skandals...,http://www.tz.de/stars/der-skandalstar-ist-lae...,0.992188
7,http://www.az-online.de/boulevard/der-skandals...,https://www.tageblatt.de/Nachrichten/Der-Skand...,0.945312
8,http://www.az-online.de/deutschland/spd-droht-...,http://www.hna.de/welt/spd-droht-wegen-afd-kom...,0.992188
9,http://www.az-online.de/deutschland/spd-droht-...,http://www.kreisbote.de/welt/spd-droht-wegen-a...,0.992188


In [ ]:


import json

# Save the duplicate pairs DataFrame to a JSON file (list of records)
duplicates_df.to_json("duplicates_pairs.json", orient="records", lines=False)

# Save the duplicates mapping (each article -> similar articles) to a JSON file
with open("duplicates_mapping.json", "w") as f:
    json.dump(duplicates_found, f)

print("Output saved to 'duplicates_pairs.json' and 'duplicates_mapping.json'.")


Output saved to 'duplicates_pairs.json' and 'duplicates_mapping.json'.


In [ ]:

# Build adjacency list for near-duplicate pairs (undirected graph)
adjacency = {}
for id1, id2 in pairs:
    adjacency.setdefault(id1, set()).add(id2)
    adjacency.setdefault(id2, set()).add(id1)

# Traverse the graph to get connected components (clusters of similar articles)
clusters = []
visited = set()
for node in adjacency:
    if node not in visited:
        cluster_set = set()
        stack = [node]
        visited.add(node)
        while stack:
            n = stack.pop()
            cluster_set.add(n)
            for neigh in adjacency[n]:
                if neigh not in visited:
                    visited.add(neigh)
                    stack.append(neigh)
        clusters.append(cluster_set)

# Filter out clusters of size 1 (articles with no near-duplicates)
clusters = [c for c in clusters if len(c) > 1]

# Convert each set to a sorted list for JSON serialization
clusters_list = [sorted(list(c)) for c in clusters]

# Save the clusters of similar articles to a JSON file
with open("duplicates_clusters.json", "w") as f:
    json.dump(clusters_list, f)

print(f"Total clusters of near-duplicates: {len(clusters_list)}")
print("Output saved to 'duplicates_clusters.json'.")


Total clusters of near-duplicates: 806
Output saved to 'duplicates_clusters.json'.
